In [1]:
import pandas as pd
import glob
import os
import numpy as np
import datetime as datetime

In [2]:
# importing libraries
import pandas as pd
import glob
import os

# merging the files
joined_files = os.path.join(r"C:\Users\kogla\OneDrive\Masaüstü\SWE599\dataset1", "*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True).drop('Unnamed: 0', axis=1)

df.head()

,checked_date,company,departure_airport,arrival_airport,departure_time,arrival_time,departure_date,duration,price
0,15/10/2022 23:02,Pegasus,SAW,ADA,16:00,17:25,"01 Ara 2022, Per",1sa 25dk,"702,99"
1,15/10/2022 23:02,Pegasus,SAW,ADA,07:45,09:10,"01 Ara 2022, Per",1sa 25dk,"742,99"
2,15/10/2022 23:02,Pegasus,SAW,ADA,20:20,21:45,"01 Ara 2022, Per",1sa 25dk,"742,99"
3,15/10/2022 23:02,Pegasus,SAW,ADA,11:40,13:05,"01 Ara 2022, Per",1sa 25dk,"792,99"
4,15/10/2022 23:02,Pegasus,SAW,ADA,18:10,19:35,"01 Ara 2022, Per",1sa 25dk,"792,99"


In [3]:
df[['departure_date','departure_week_day']] = df['departure_date'].str.split(",",expand=True)

In [4]:
df['departure_date'] = df['departure_date'].str.replace(' Ara ', '/12/')
df['departure_date'] = df['departure_date'].str.replace(' Kas ', '/11/')
df['departure_date'] = df['departure_date'].str.replace(' Eki ', '/10/')
df['departure_date'] = df['departure_date'].str.replace(' Eyl ', '/09/')
df['departure_date'] = df['departure_date'].str.replace(' Ağu ', '/08/')
df['departure_date'] = df['departure_date'].str.replace(' Tem ', '/07/')
df['departure_date'] = df['departure_date'].str.replace(' Haz ', '/06/')
df['departure_date'] = df['departure_date'].str.replace(' May ', '/05/')
df['departure_date'] = df['departure_date'].str.replace(' Nis ', '/04/')
df['departure_date'] = df['departure_date'].str.replace(' Mar ', '/03/')
df['departure_date'] = df['departure_date'].str.replace(' Şub ', '/02/')
df['departure_date'] = df['departure_date'].str.replace(' Oca ', '/01/')
df['departure_date'] = df['departure_date'].str.replace(' ', '')

In [5]:
df[['checked_date','checked_time']] = df['checked_date'].str.split(" ",expand=True)

In [6]:
# str to datetime conversion for dates DD-MM-YY
df['departure_date'] = pd.to_datetime(df['departure_date'], format='%d/%m/%Y')
df['checked_date'] = pd.to_datetime(df['checked_date'], format='%d/%m/%Y')

# str to datetime conversion for time HH:MM:SS
df['departure_time'] = pd.to_datetime(df['departure_time'],format= '%H:%M' ).dt.time
df['arrival_time'] = pd.to_datetime(df['arrival_time'],format= '%H:%M' ).dt.time
df['checked_time'] = pd.to_datetime(df['checked_time'],format= '%H:%M' ).dt.time

# week day calculation
df['departure_week_day'] = df['departure_date'].dt.day_name()
df['checked_week_day'] = df['checked_date'].dt.day_name()

In [7]:
# price str to float conversion
df.columns = df.columns.str.replace('price', 'price_try')
df['price_try'] = df['price_try'].str.replace('.', '', regex=False)
df['price_try'] = df['price_try'].str.replace(",", ".", regex=False)
df["price_try"] = pd.to_numeric(df["price_try"], downcast="float")
df["price_try"]=df["price_try"].apply(lambda row: round(row,2))

In [8]:
# calculation remaining days to flight
df['remaining_day_to_flight'] = (df['departure_date'] - df['checked_date']).dt.days

In [9]:
def arrival_city_name(row):
    if row == "ADB" :
        return 'Izmir'
    elif row == "ESB" :
        return 'Ankara'
    elif row== "AYT" :
        return 'Antalya'
    elif row == "ADA" :
        return 'Adana'
    elif row == "BJV" :
        return 'Bodrum Milas'
    elif row == "DLM" :
        return 'Dalaman'
    elif row == "TZX" :
        return 'Trabzon'
    elif row == "GZT" :
        return 'Gaziantep'
    elif row == "DIY" :
        return 'Diyarbakir'
    elif row == "HTY" :
        return 'Hatay'
    elif row == "KSY" :
        return 'Kars'
    elif row == "ERZ" :
        return 'Erzurum'
    elif row == "MQM" :
        return 'Mardin'
    else:
        return none

In [10]:
def departure_airport_name(row):
    if row == "SAW" :
        return 'Sabiha Gökçen Airport'
    elif row == "IST" :
        return 'Istanbul Airport'
    else:
        return none

In [11]:
# create to column for city names
df["arrival_city"]=df["arrival_airport"].apply(lambda row: arrival_city_name(row))
df["departure_airport_name"]=df["departure_airport"].apply(lambda row: departure_airport_name(row))

In [12]:
# calculate day part info for flight departure time
df['departure_hour'] = df.departure_time.apply(lambda x: x.hour)
conditions = [df.departure_hour.between(0, 6), df.departure_hour.between(6, 12), df.departure_hour.between(12, 17), df.departure_hour.between(17,21), df.departure_hour.between(21,24)]
choices = ['Late Night', 'Morning', 'Afternoon', 'Evening', 'Night']
df['part_of_day'] = np.select(conditions, choices)
df.drop('departure_hour', axis=1)

,checked_date,company,departure_airport,arrival_airport,departure_time,arrival_time,departure_date,duration,price_try,departure_week_day,checked_time,checked_week_day,remaining_day_to_flight,arrival_city,departure_airport_name,part_of_day
0,2022-10-15,Pegasus,SAW,ADA,16:00:00,17:25:00,2022-12-01,1sa 25dk,702.99,Thursday,23:02:00,Saturday,47,Adana,Sabiha Gökçen Airport,Afternoon
1,2022-10-15,Pegasus,SAW,ADA,07:45:00,09:10:00,2022-12-01,1sa 25dk,742.99,Thursday,23:02:00,Saturday,47,Adana,Sabiha Gökçen Airport,Morning
2,2022-10-15,Pegasus,SAW,ADA,20:20:00,21:45:00,2022-12-01,1sa 25dk,742.99,Thursday,23:02:00,Saturday,47,Adana,Sabiha Gökçen Airport,Evening
3,2022-10-15,Pegasus,SAW,ADA,11:40:00,13:05:00,2022-12-01,1sa 25dk,792.99,Thursday,23:02:00,Saturday,47,Adana,Sabiha Gökçen Airport,Morning
4,2022-10-15,Pegasus,SAW,ADA,18:10:00,19:35:00,2022-12-01,1sa 25dk,792.99,Thursday,23:02:00,Saturday,47,Adana,Sabiha Gökçen Airport,Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5257,2022-10-15,Türk Hava Yolları,IST,TZX,09:40:00,11:25:00,2022-11-30,1sa 45dk,1007.99,Wednesday,22:59:00,Saturday,46,Trabzon,Istanbul Airport,Morning
5258,2022-10-15,Türk Hava Yolları,IST,TZX,14:10:00,15:55:00,2022-11-30,1sa 45dk,1007.99,Wednesday,22:59:00,Saturday,46,Trabzon,Istanbul Airport,Afternoon
5259,2022-10-15,Türk Hava Yolları,IST,TZX,19:30:00,21:15:00,2022-11-30,1sa 45dk,1007.99,Wednesday,22:59:00,Saturday,46,Trabzon,Istanbul Airport,Evening
5260,2022-10-15,Türk Hava Yolları,IST,TZX,17:15:00,19:05:00,2022-11-30,1sa 45dk,1007.99,Wednesday,22:59:00,Saturday,46,Trabzon,Istanbul Airport,Afternoon
